In [4]:
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [5]:
df = pd.read_csv('train.txt', sep=" ")
# no-need for the third column
df = df['POS-Tag']

In [6]:
CONTEXT_SIZE = 6
EMBEDDING_DIM = 10
# preparing the training data
trigrams = [([df[i], df[i + 1], df[i + 2], df[i + 3], df[i + 4], df[i + 5]], df[i + 6])
            for i in range(len(df) - 6)]
vocab = set(df) # this contains special characters also
word_to_ix = {word: i for i, word in enumerate(vocab)}


In [7]:
# architecture of the NN

class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs



In [9]:
#training


losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(2):
    total_loss = 0
    for context, target in trigrams:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!


[521429.8845413625, 479532.32176690875]


In [11]:
# preparing the data for testing
df_test = pd.read_csv('test.txt', sep=" ")
# no-need for the third column
df_test = df_test['POS-Tag']

# preparing the training data
trigrams_test = [([df[i], df[i + 1], df[i + 2], df[i + 3], df[i + 4], df[i + 5]], df[i + 6])
            for i in range(len(df) - 6)]


In [14]:
print(len(trigrams_test))

211721


In [25]:
# testing



model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)

# ans.csv file contains whether a word is considered to be correct or not
f = open('ans.txt', 'a')

for context, target in trigrams_test:

    # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
    # into integer indices and wrap them in tensors)
    context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)

    # Step 2. Recall that torch *accumulates* gradients. Before passing in a
    # new instance, you need to zero out the gradients from the old
    # instance
#         model.zero_grad()

    # Step 3. Run the forward pass, getting log probabilities over next
    # words
    log_probs = model(context_idxs)
#     print('Context words: {}, Target = {}'.format(context, target))
#     print(log_probs)

    # Step 4. Compute your loss function. (Again, Torch wants the target
    # word wrapped in a tensor)
    loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))
#     print('target = {}\tloss = {}'.format(target, loss))
    if loss >= 3.5:
        f.write('1\n')
    else:
        f.write('0\n')
    f.flush()
    
f.close()
print('completed writing the answers"')




completed writing the answers"
